In [5]:
import os
import cv2
import numpy as np
from PIL import Image, ImageSequence, UnidentifiedImageError
from ipywidgets import widgets
from tkinter import Tk, filedialog, messagebox
from scipy.ndimage import gaussian_filter, median_filter

# Global variables for selected file and directory
input_file = None
output_dir = None

# Function to select a file using tkinter
def select_file(prompt="Select .tif File"):
    """
    Opens a file dialog for the user to select a .tif file. If a file has already been selected, it returns that file.

    Args:
        prompt (str): The prompt displayed in the file selection dialog.

    Returns:
        str: The file path of the selected file.
    """
    global input_file
    if not input_file:  # Select file only if not already selected
        root = Tk()
        root.withdraw()  # Hide the root window
        input_file = filedialog.askopenfilename(title=prompt, filetypes=[("TIFF files", "*.tif")])
        root.update()
    return input_file

# Function to select a directory using tkinter
def select_directory(prompt="Select Directory"):
    """
    Opens a directory selection dialog for the user to choose a folder. If a folder has already been selected, it returns that folder.

    Args:
        prompt (str): The prompt displayed in the directory selection dialog.

    Returns:
        str: The path of the selected directory.
    """
    global output_dir
    if not output_dir:  # Select directory only if not already selected
        root = Tk()
        root.withdraw()  # Hide the root window
        output_dir = filedialog.askdirectory(title=prompt)
        root.update()
    return output_dir

# Apply 3D Gaussian blur to a 3D image with different radii for x, y, and z axes
def apply_3d_gaussian_blur(image_array, blur_radius_x, blur_radius_y, blur_radius_z):
    """
    Applies a 3D Gaussian blur to the input image array using different blur radii for x, y, and z axes.

    Args:
        image_array (np.ndarray): The 3D image data as a numpy array.
        blur_radius_x (float): Gaussian blur radius along the x-axis.
        blur_radius_y (float): Gaussian blur radius along the y-axis.
        blur_radius_z (float): Gaussian blur radius along the z-axis.

    Returns:
        np.ndarray: The blurred image array, clipped to the range [0, 255] and converted to uint8 format.
    """
    blurred = gaussian_filter(image_array, sigma=(blur_radius_z, blur_radius_y, blur_radius_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Median blur
def apply_3d_median_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    """
    Applies a 3D median blur to the input image array using a specified kernel size for the x, y, and z axes.

    Args:
        image_array (np.ndarray): The 3D image data as a numpy array.
        kernel_size_x (int): The size of the median filter kernel along the x-axis.
        kernel_size_y (int): The size of the median filter kernel along the y-axis.
        kernel_size_z (int): The size of the median filter kernel along the z-axis.

    Returns:
        np.ndarray: The blurred image array, clipped to the range [0, 255] and converted to uint8 format.
    """
    blurred = median_filter(image_array, size=(kernel_size_z, kernel_size_y, kernel_size_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Mean blur
def apply_3d_mean_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    """
    Applies a 3D mean blur to the input image array using a specified kernel size for the x, y, and z axes.

    Args:
        image_array (np.ndarray): The 3D image data as a numpy array.
        kernel_size_x (int): The size of the mean filter kernel along the x-axis.
        kernel_size_y (int): The size of the mean filter kernel along the y-axis.
        kernel_size_z (int): The size of the mean filter kernel along the z-axis (unused in this function).

    Returns:
        np.ndarray: The blurred image array, clipped to the range [0, 255] and converted to uint8 format.
    """
    blurred = cv2.blur(image_array, (kernel_size_x, kernel_size_y))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Save processed images (frames) as a multi-page TIFF
def save_image(frames, output_dir, file_name, suffix):
    """
    Saves a series of frames as a multi-page TIFF file.

    Args:
        frames (list of PIL.Image): List of PIL Image objects representing individual frames of the 3D image.
        output_dir (str): The directory where the output TIFF file will be saved.
        file_name (str): The base name of the output file, without extension.
        suffix (str): A suffix to append to the file name, indicating the type of blur applied.

    Returns:
        None
    """
    output_path = os.path.join(output_dir, f"{file_name}_{suffix}.tif")
    frames[0].save(output_path, save_all=True, append_images=frames[1:])
    print(f"Saved multi-page TIFF: {output_path}")

# Process 3D TIFF file for blurring with enhanced error handling
def process_file_blur(file_path, output_dir, blur_type, blur_radius_x=0, blur_radius_y=0, blur_radius_z=0, kernel_size=3):
    """
    Processes a 3D TIFF file by applying a specified blur type (Gaussian, Mean, or Median) and saves the result as a TIFF.

    Args:
        file_path (str): The path to the input .tif file.
        output_dir (str): The directory where the output blurred TIFF will be saved.
        blur_type (str): The type of blur to apply ("Gaussian", "Mean", or "Median").
        blur_radius_x (float, optional): The blur radius along the x-axis for Gaussian blur. Defaults to 0.
        blur_radius_y (float, optional): The blur radius along the y-axis for Gaussian blur. Defaults to 0.
        blur_radius_z (float, optional): The blur radius along the z-axis for Gaussian blur. Defaults to 0.
        kernel_size (int, optional): The kernel size for Median and Mean blur. Defaults to 3.

    Returns:
        None
    """
    try:
        with Image.open(file_path) as img:
            frames = [np.array(frame) for frame in ImageSequence.Iterator(img)]
            image_3d_array = np.stack(frames)  # Stack frames into a 3D array (Z, Y, X)
            print(f"Loaded 3D image with shape: {image_3d_array.shape}")

            if blur_type == "Gaussian":
                blurred_image_3d = apply_3d_gaussian_blur(image_3d_array, blur_radius_x, blur_radius_y, blur_radius_z)
            elif blur_type == "Median":
                blurred_image_3d = apply_3d_median_blur(image_3d_array, kernel_size, kernel_size, kernel_size_z=kernel_size)
            elif blur_type == "Mean":
                blurred_image_3d = apply_3d_mean_blur(image_3d_array, kernel_size, kernel_size, kernel_size)

            blurred_frames = [Image.fromarray(blurred_image_3d[i, :, :]) for i in range(blurred_image_3d.shape[0])]
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            save_image(blurred_frames, output_dir, file_name, f"{blur_type.lower()}_blur")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

# Start blurring function
def start_blurring(blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size):
    """
    Initiates the blurring process based on the selected blur type and parameters.

    Args:
        blur_type (str): The type of blur to apply ("Gaussian", "Mean", or "Median").
        blur_radius_x (float): The blur radius along the x-axis (for Gaussian blur).
        blur_radius_y (float): The blur radius along the y-axis (for Gaussian blur).
        blur_radius_z (float): The blur radius along the z-axis (for Gaussian blur).
        kernel_size (int): The size of the kernel (for Mean and Median blurs).

    Returns:
        None
    """
    if input_file and output_dir:
        process_file_blur(input_file, output_dir, blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size)
        print("Blurring completed.")
    else:
        print("Please select input file and output directory first.")

# Widget for selecting file and directories
file_dir_selector = widgets.Button(description="Select File and Directory")
file_dir_selector.on_click(lambda x: (select_file(), select_directory()))

# Widgets for blurring
blur_type_dropdown = widgets.Dropdown(
    options=["Gaussian", "Mean", "Median"],
    value="Gaussian",
    description="Blur Type:"
)

blur_x_input = widgets.IntText(value=3, description="Blur Radius X")
blur_y_input = widgets.IntText(value=3, description="Blur Radius Y")
blur_z_input = widgets.IntText(value=1, description="Blur Radius Z")  # Z-axis control
kernel_size_input = widgets.IntText(value=3, description="Kernel Size", layout=widgets.Layout(display='none'))

blur_button = widgets.Button(description="Start Blurring")
blur_button.on_click(lambda x: start_blurring(
    blur_type_dropdown.value,
    blur_x_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_y_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_z_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    kernel_size_input.value if blur_type_dropdown.value in ["Mean", "Median"] else 3
))

# Function to show/hide kernel size based on blur type
def update_kernel_size_visibility(change):
    """
    Updates the visibility of the kernel size input widget based on the selected blur type.

    If "Gaussian" is selected, the kernel size input is hidden.
    If "Mean" or "Median" is selected, the kernel size input is shown.

    Args:
        change (dict): Contains information about the dropdown change event.
    """
    if change['new'] == "Gaussian":
        kernel_size_input.layout.display = 'none'
    else:
        kernel_size_input.layout.display = 'block'

blur_type_dropdown.observe(update_kernel_size_visibility, names='value')

# Display widgets
display(file_dir_selector, blur_type_dropdown, blur_x_input, blur_y_input, blur_z_input, kernel_size_input, blur_button)

Button(description='Select File and Directory', style=ButtonStyle())

Dropdown(description='Blur Type:', options=('Gaussian', 'Mean', 'Median'), value='Gaussian')

IntText(value=3, description='Blur Radius X')

IntText(value=3, description='Blur Radius Y')

IntText(value=1, description='Blur Radius Z')

IntText(value=3, description='Kernel Size', layout=Layout(display='none'))

Button(description='Start Blurring', style=ButtonStyle())

In [2]:
import os
import cv2
import numpy as np
from PIL import Image, ImageSequence, UnidentifiedImageError
from ipywidgets import widgets
from tkinter import Tk, filedialog, messagebox
from scipy.ndimage import gaussian_filter, median_filter

# Global variables for selected file and directory
input_file = None
output_dir = None

# Function to select a file using tkinter
def select_file(prompt="Select .tif File"):
    global input_file
    if not input_file:  # Select file only if not already selected
        root = Tk()
        root.withdraw()  # Hide the root window
        input_file = filedialog.askopenfilename(title=prompt, filetypes=[("TIFF files", "*.tif")])
        root.update()
    return input_file

# Function to select directory using tkinter
def select_directory(prompt="Select Directory"):
    global output_dir
    if not output_dir:  # Select directory only if not already selected
        root = Tk()
        root.withdraw()  # Hide the root window
        output_dir = filedialog.askdirectory(title=prompt)
        root.update()
    return output_dir

# Apply 3D Gaussian blur to a 3D image with different radii for x, y, and z axes
def apply_3d_gaussian_blur(image_array, blur_radius_x, blur_radius_y, blur_radius_z):
    blurred = gaussian_filter(image_array, sigma=(blur_radius_z, blur_radius_y, blur_radius_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Median blur
def apply_3d_median_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    blurred = median_filter(image_array, size=(kernel_size_z, kernel_size_y, kernel_size_x))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Apply 3D Mean blur
def apply_3d_mean_blur(image_array, kernel_size_x, kernel_size_y, kernel_size_z):
    blurred = cv2.blur(image_array, (kernel_size_x, kernel_size_y))
    return np.clip(blurred, 0, 255).astype(np.uint8)

# Save processed images (frames) as a multi-page TIFF
def save_image(frames, output_dir, file_name, suffix):
    output_path = os.path.join(output_dir, f"{file_name}_{suffix}.tif")
    frames[0].save(output_path, save_all=True, append_images=frames[1:])
    print(f"Saved multi-page TIFF: {output_path}")

# Process 3D TIFF file for blurring with enhanced error handling
def process_file_blur(file_path, output_dir, blur_type, blur_radius_x=0, blur_radius_y=0, blur_radius_z=0, kernel_size=3):
    try:
        with Image.open(file_path) as img:
            frames = [np.array(frame) for frame in ImageSequence.Iterator(img)]
            image_3d_array = np.stack(frames)  # Stack frames into a 3D array (Z, Y, X)
            print(f"Loaded 3D image with shape: {image_3d_array.shape}")

            if blur_type == "Gaussian":
                blurred_image_3d = apply_3d_gaussian_blur(image_3d_array, blur_radius_x, blur_radius_y, blur_radius_z)
            elif blur_type == "Median":
                blurred_image_3d = apply_3d_median_blur(image_3d_array, kernel_size, kernel_size, kernel_size_z=kernel_size)
            elif blur_type == "Mean":
                blurred_image_3d = apply_3d_mean_blur(image_3d_array, kernel_size, kernel_size, kernel_size)

            blurred_frames = [Image.fromarray(blurred_image_3d[i, :, :]) for i in range(blurred_image_3d.shape[0])]
            file_name = os.path.splitext(os.path.basename(file_path))[0]
            save_image(blurred_frames, output_dir, file_name, f"{blur_type.lower()}_blur")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

# Start blurring function
def start_blurring(blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size):
    if input_file and output_dir:
        process_file_blur(input_file, output_dir, blur_type, blur_radius_x, blur_radius_y, blur_radius_z, kernel_size)
        print("Blurring completed.")
    else:
        print("Please select input file and output directory first.")

# Widget for selecting file and directories
file_dir_selector = widgets.Button(description="Select File and Directory")
file_dir_selector.on_click(lambda x: (select_file(), select_directory()))

# Widgets for blurring
blur_type_dropdown = widgets.Dropdown(
    options=["Gaussian", "Mean", "Median"],
    value="Gaussian",
    description="Blur Type:"
)

blur_x_input = widgets.IntText(value=3, description="Blur Radius X")
blur_y_input = widgets.IntText(value=3, description="Blur Radius Y")
blur_z_input = widgets.IntText(value=1, description="Blur Radius Z")  # Z-axis control
kernel_size_input = widgets.IntText(value=3, description="Kernel Size", layout=widgets.Layout(display='none'))

blur_button = widgets.Button(description="Start Blurring")
blur_button.on_click(lambda x: start_blurring(
    blur_type_dropdown.value,
    blur_x_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_y_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    blur_z_input.value if blur_type_dropdown.value == "Gaussian" else 0,
    kernel_size_input.value if blur_type_dropdown.value in ["Mean", "Median"] else 3
))

# Function to show/hide kernel size based on blur type
def update_kernel_size_visibility(change):
    if change['new'] == "Gaussian":
        kernel_size_input.layout.display = 'none'
    else:
        kernel_size_input.layout.display = 'block'

blur_type_dropdown.observe(update_kernel_size_visibility, names='value')

# Display widgets
display(file_dir_selector, blur_type_dropdown, blur_x_input, blur_y_input, blur_z_input, kernel_size_input, blur_button)

Button(description='Select File and Directory', style=ButtonStyle())

Dropdown(description='Blur Type:', options=('Gaussian', 'Mean', 'Median'), value='Gaussian')

IntText(value=3, description='Blur Radius X')

IntText(value=3, description='Blur Radius Y')

IntText(value=1, description='Blur Radius Z')

IntText(value=3, description='Kernel Size', layout=Layout(display='none'))

Button(description='Start Blurring', style=ButtonStyle())